This is a notebook for the Week 3 of the Developer Challenge: https://blogs.sap.com/2023/03/08/sap-community-developer-challenge-eda-with-sap-hana-and-python/

Some code might be incomplete or incorrect. It is your task to fix it and execute the whole notebook receiving correct results.

Submit the link to your solution: https://groups.community.sap.com/t5/application-development/submissions-for-quot-eda-with-sap-hana-quot-developer-challenge/m-p/225900/highlight/true#M1181

## Initialize the notebook

Import required packages

In [ ]:
from hana_ml import dataframe as hdf
import pandas as pd

Connect to SAP HANA database instance with a database user, same as during Week 2!

In [ ]:
myconn=hdf.ConnectionContext(userkey='DevChallenger')
print("SAP HANA DB version: ", myconn.hana_ver())

In [ ]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

## Check tables from Week 2 are available for this week's exercises

You should see these 3 tables listed: `BIKES`, `MONTHS`, `SEASONS`. If not, then please complete [the Week 2](https://github.com/SAP-samples/sap-community-developer-challenge-eda-hana/blob/main/notebooks/challenges/week2.ipynb) first.

In [ ]:
myconn.get_tables()

Previous week you created HANA DataFrame `bikes_hdf` at the moment of loading a CSV file. 

Now the table with data should already exist in your SAP HANA database, so you use [the `table()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.ConnectionContext.table) this time to create a HANA DataFrame from an existing database table. 

In [ ]:
bikes_hdf=myconn.table('BICYCLES')

In [ ]:
bikes_hdf.select_statement

## 1. Understand the dataset

Before you understand the data, you need to understand the dataset: what kind of data is stores and its size.

In [ ]:
print(f"DataFrame returns {bikes_hdf.count()} record(s)")

But if you are familiar with Pandas dataframes, then typically you want to understand both: numbers of rows and columns in a dataframe. With HANA DataFrame you use [an attribute `shape`](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.shape) for the same.

In [ ]:
bikes_hdf.shape()

So, there are 12 columns. [The property `columns`](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.columns) returns their names.

In [ ]:
bikes_hdf.columns

To get SQL types of the columns use [the method `dtypes()`](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.dtypes)...

In [ ]:
bikes_hdf.dtypes()

...for example to return a list of columns, which SQL data type is [`NVARCHAR`](https://help.sap.com/docs/HANA_CLOUD_DATABASE/c1d3f60099654ecfb3fe36ac93c121bb/a33f7884b0c14c00b1a76ecd8af5feca.html?locale=en-US&version=2022_4_QRC).

In [ ]:
[col[0] for col in bikes_hdf.datatypes() if col[1]=='NVARCHAR']

In the next step you usally want to look at a few records of the data. At the end of the notebook from Week 2 you used [Pandas method `head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html#pandas.DataFrame.head) to return ten first records from the Pandas DataFrame...

In [ ]:
bikes_hdf.collect().head(10)

...but this method was applied after `collect()`, when **all** 731 records from HANA table where read into a Python client. With a few hundreds of records it was still Ok, but it would be expensive with tables containing hundreds of thousands of records.

When working with the data stored in a database you should push processing down to the database and have only results returned to the client.

HANA DataFrame has [similar `head()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.head)...

In [ ]:
bikes_hdf.head(10).collect()

...but this time these 10 records are selected from a table already in the database using `TOP X` syntax of SQL `SELECT` statement; and only these 10 records are returned to the client.

In [ ]:
bikes_hdf.head(10).sql_statement

Similarly to Pandas a HANA DataFrame provides [a `tail()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.tail), but allows you to provide a reference column `ref_col` accordingly to which a dataset should be sorted before the last X (in this example **5**) records are selected in the database and returned to a client.

In [ ]:
bikes_hdf.tail(5, ref_col='days_since_2012').collect()

As `SELECT` statement in SQL does not provide equivalent to `TOP` syntax to return bottom records, the SELECT statement generated for you by `hana-ml` package is more complex than the one for the `head()`.

In [ ]:
bikes_hdf.tail(5, ref_col='days_since_2011').select_statement

Typically for Machine Learning scenarios you want to analyze, if a dataset contains `NULL` values, which might decrease the quality of data analysis or ML models created. Use [a method `hasna()`](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.hasna) to check NULL values in a HANA DataFrame...

In [ ]:
bikes_hdf.hasna()

...or in particular DataFrame's columns:

In [ ]:
for col in bikes_hdf.columns:
    print(f"NULLs in {col}: {bikes_hdf.hasnull(col)}")

Generate descriptive statistics in database. [A method `describe()`](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.describe) returns a DataFrame that contains various statistics for the requested column(s) or the complete DataFrame.

In [ ]:
bikes_hdf.describe().collect()

A method `describe()` returns a HANA DataFrame, that's why you had to use `collect()` in the previous cell.

You can see an SQL statement of the HANA Frame generated by the `describe()`.

In [ ]:
bikes_hdf.describe().select_statement

LEt's get back to the results returned by `bikes_hdf.describe().collect()` above.

As tested before with `hasna()` method, you can confirm there are no NULL values in any column, because `nulls` column in the describe's result contains only `0`. 

### Closer look at properties of columns in a dataframe

Check statistics only for columns storing strings (SQL data type `NVARCHAR`).

In [ ]:
bikes_hdf.describe([col[0] for col in bikes_hdf.dtypes() if col[1]=='NVARCHAR']).collect().dropna(axis=1,how='all')

To increase a readability of such long lines of chained methods it is good to follow the recommendation:
> Long lines can be broken over multiple lines by wrapping expressions in parentheses.

from [PEP 8 "Style Guide for Python Code"](https://peps.python.org/pep-0008/#maximum-line-length).

In [ ]:
(bikes_hdf
 .describe([col[0] for col in bikes_hdf.dtypes() if col[1]='NVARCHAR'])
 .collect()
# .dropna(axis=1,how='all') # uncomment this line to see the difference
 .set_index('column')
)

There are 4 unique values in the `season` column. Check them using [a `distinct()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.distinct), and -- as expected -- you should get four season names.

In [ ]:
bikes_hdf.distinct(cols='season').collect()

As cardinality of each of those columns storing stings is low (12 unique values at the most). Let's list all values for all of those columns.

In [ ]:
for column in [col[0] for col in bikes_hdf.dtypes() if col[1]=='NVARCHAR']:
    print(f"{column}: {bikes_hdf.distinct(column).collect().iloc[:,0].tolist()}")

For example, there are only 3 values in `weathersit` describig a weather situation.

Check how often each weather situation occurs in the dataframe using [a `value_counts()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.value_counts). Note that it can only accept a list `[]` of column, even if the list contains only one column.

In [ ]:
bikes_hdf.value_counts(['weathersit']).collect()

Now check basic characteristics of numeric values in columns.

In [ ]:
(bikes_hdf
 .describe([col[0] for col in bikes_hdf.dtypes() if col[1]!='NVARCHAR'])
 .collect()
 .iloc[:,:9]
 .set_index('column')
)

What you can read from these values?

1. Year column `yr` contains only 2 different values. Min is `2011` and max is `2012`, so the dataframe contains data for two years: 2011 and 2012.
1. A column `days_since_2011` contains 731 unique values in 731 records, so each record represents data for one day.
1. Min value for that column is `0` and the max is `730`, so we have values for 731 consequtive days, starting from 2011-01-01.
1. A column `cnt` stores a number of bike rentals per day.
1. A temperature `temp` range between -5.2 and 32.5 should be in degrees Celcius.
1. A `hum` is a humidity percentage, although min value of 0% must be a mistake in data.| 

In [ ]:
bikes_hdf.value_counts(subset='yr').collect()

For continues, and not discrete, values (like `hum`) binning is more appropriate in the process of data exploration. Use [a `bin` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.bin) for that.

In [ ]:
(bikes_hdf
 .select('hum').sort('hum')
 .bin('hum', strategy='uniform_numbers', bins=10)
 .collect()
)

Indeed `0.0000` humidity percentage should be an outlier, for example wrong data recorded...

In [ ]:
(bikes_hdf
 .select('hum').sort('hum')
 .bin('hum', strategy='uniform_number', bins=10)
 .value_counts(subset=['BIN_NUMBER'])
 .collect().set_index('VALUES')
)

...and it is the only value in the 1st bin. Most of the values are in bins 6, 7 and 8 (so between 50% and 80% humidity).

It would be the best to support such analysis visually, but **this is the topic for next Week 4**.

## 2. Understand the data

Let's look at how HANA DataFrames support filtering (selecting rows), projection (selecting columns), and joining (combining datasets) -- basic [relational operations](https://en.wikipedia.org/wiki/Relational_model#Relational_operations) on tabular data stored in SAP HANA tables.

### Filtering

Use [a `filter()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.filter) to return only rows that match the given condition.

A `condition` parameter is included into the generated SQL SELECT statement, so you need to pay attention to names of [identifiers](https://help.sap.com/docs/HANA_CLOUD_DATABASE/c1d3f60099654ecfb3fe36ac93c121bb/209f5020751910148fd8fe88aa4d79d9.html#identifiers) (here: column names). In SAP HANA database:
> "[*Identifiers without double-quotes in SQL syntax are converted to upper case when processed by the server.*](https://help.sap.com/docs/HANA_CLOUD_DATABASE/c1d3f60099654ecfb3fe36ac93c121bb/209f5020751910148fd8fe88aa4d79d9.html#identifiers-and-case-sensitivity)"

In [ ]:
bikes_hdf.filter(condition='"yr"=2011').count()

You need to be careful with quotation marks used to delimit Python strings and used for SAP HANA SQL within Python strings.

In Python:
> "[*Both types of literals can be enclosed in matching single quotes (') or double quotes ("). ... The backslash (\) character is used to escape characters that otherwise have a special meaning, such as newline, backslash itself, or the quote character.*](https://docs.python.org/3/reference/lexical_analysis.html#string-and-bytes-literals)"

In SAP HANA database:
> "[*Single quotation marks are used to delimit string literals. ... Double quotation marks are used to delimit identifiers. ...*](https://help.sap.com/docs/HANA_CLOUD_DATABASE/c1d3f60099654ecfb3fe36ac93c121bb/209f5020751910148fd8fe88aa4d79d9.html#quotation-marks)"

In [ ]:
bikes_hdf.filter('yr=2011 AND "holiday"=\'HOLIDAY\'').collect()

### Projection

Use [a `select()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.select) to return only columns you want to have in the result HANA DataFrame.

In [ ]:
(bikes_hdf
 .filter('"yr"=2011 AND "holiday"='HOLIDAY'')
 .select('weekday', 'days_since_2011')
 .collect()
)

A `select()` method can be used to derive new columns from existing when provided as a tuple `(<SQL expression>, <alias>)`.

In the example below a new column `calday` is calculated using [the SAP HANA SQL `ADD_DAYS` function](https://help.sap.com/docs/HANA_CLOUD_DATABASE/c1d3f60099654ecfb3fe36ac93c121bb/20da5fa8751910148969da2572b25ed8.html?version=2022_4_QRC&locale=en-US) to add number from the `"days_since_2011"` column to the date `2011-01-01`.

In [ ]:
(bikes_hdf
 .filter('"yr"=2011 AND "holiday"=\'HOLIDAY\'')
 .select('weekday', ('ADD_DAYS(\'2011-01-01\',"days_since_2011")', 'calday'))
 .collect()
)

Based on the holiday dates returned you can deduct they are [US Federal Holidays 2011](https://www.calendarpedia.com/holidays/federal-holidays-2011.html).

Note that if you create a new HANA DataFrame `bikebydays_hdf` that has all columns (`'*'`) from the original dataframe `bikes_hdf` plus a derived column `calday`, no database object is created in SAP HANA.

In [ ]:
bikebydays_hdf=bikes_hdf.select('*', ('add_days(\'2011-01-01\',"days_since_2011")', 'calday'))

This HANA DataFrame exists only in the scope of a Python session and stores an SQL SELECT statement used to query data from its connection to an SAP HANA database.

In [ ]:
bikebydays_hdf.select_statement

Use this HANA DataFrame `bikebydays_hdf` to query holidays in 2012.

In [ ]:
(bikebydays_hdf
 .filter('"yr"=2012 AND "holiday"=\'HOLIDAY\'')
 .select('weekday', 'caldate')
 .collect()
)

Find days with the smallest numbers of bike rentals.

In [ ]:
bikebydays_hdf.deselect('days_since_2011').sort(cols='cnt', desc=False).head(5).collect()

What is the smallest count of bikes rented during one day?

`Edit this markdown cell and type your answer:`

### Joining

Let's check summer months of 2012.

Please note tripple single quotes used by Python string. You do not need to use escape character `\` in front of single qoutes of HANA's string of `'SUMMER'` in such case.

In [ ]:
bikes_hdf.filter('''
"season"='SUMMER' AND "yr"=2012
''').distinct('mnth').collect()

In [ ]:
(bikebydays_hdf.filter('''
"season"='SUMMER' AND "yr"=2012
''')
 .agg(
     [('min', 'calday', 'MIN'),
      ('mix', 'calday', 'MAX')], 
     group_by=['mnth']
 )
 .collect()
)

Month are not ordered as you might used to see elsewhere. Let's use another table `MONTHS`, which you created during Week 2, to order these months accordingly to their `ID` number.

In [ ]:
myconn.table('MONTHS').collect()

In this last step this week, let's [join](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.join) last two HANA DataFrames to order month by their `ID` from `MONTHS` table.

In [ ]:
bikebydays_hdf.filter('''
"season"='SUMMER' AND "yr"=2012
''')
 .agg(
     [('min', 'calday', 'MIN'),
      ('max', 'calday', 'MAX')], 
     group_by=['mnth']
 )
 .join(other=myconn.table('MONTHS'), condition='"mnth"="DESC"', 
       select=['ID', ('DESC', 'MONTH'), 'MIN', 'MAX'])
 .sort('ID')
 .collect()
 .set_index('ID')


So, in this dataset seasons are [astronomical seasons](https://en.wikipedia.org/wiki/Season#Astronomical), where summer of 2012 started on June 21st and ended on Sep 22.

Well done! You've explored a lot of insights about data in the bikes rental dataset.

During next Week 4 -- the last week of this challenge -- you will work with `hana-ml` provided visualization of data in SAP HANA database to discover more insights!

### Stay tuned for the Week 4!